In [1]:
from nlp import load_dataset
dataset = load_dataset('de_politik_news.py', cache_dir='data')

/home/daksenov/miniconda2/envs/summ/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/daksenov/miniconda2/envs/summ/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/daksenov/miniconda2/envs/summ/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/daksenov/miniconda2/envs/summ/lib/pyt

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize
vectorizer = TfidfVectorizer(tokenizer = word_tokenize)
X_train = vectorizer.fit_transform(dataset['train']['text'])
y_train = dataset['train']['class']
X_test = vectorizer.transform(dataset['test']['text'])
y_test = dataset['test']['class']

In [4]:
from sklearn.metrics import accuracy_score, f1_score, classification_report
#class_test = dataset['test']['class']
def evaluate(model_test, class_test):
    accuracy = accuracy_score(class_test, model_test)
    f1_micro = f1_score(class_test, model_test, average = 'micro')
    f1_macro = f1_score(class_test, model_test, average = 'macro')
    report = classification_report(class_test, model_test)
    print(f'accuracy: {accuracy}')
    print(f'F1-micro: {f1_micro}')
    print(f'F1-macro: {f1_macro}')
    print(f'Report: {report}')

# Random forest

In [5]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(max_depth=30, min_impurity_decrease = 0.001, 
                               criterion = "entropy", n_estimators=1000, 
                               class_weight='balanced', random_state=0)
model = model.fit(X_train,y_train)

In [12]:
evaluate(model.predict(X_test), y_test)

accuracy: 0.4257964257964258
F1-micro: 0.4257964257964258
F1-macro: 0.43026245023451065
Report:               precision    recall  f1-score   support

      center       0.31      0.23      0.27      1349
 center-left       0.34      0.38      0.36      1159
center-right       0.51      0.55      0.53      1754
    far-left       0.59      0.40      0.48       215
   far-right       0.46      0.58      0.51       671

    accuracy                           0.43      5148
   macro avg       0.44      0.43      0.43      5148
weighted avg       0.42      0.43      0.42      5148



In [11]:
import pickle
pickle.dump(model, open('models/RF/model.bin', 'wb'))
model = pickle.load(open('models/RF/model.bin', 'rb'))

pickle.dump(vectorizer, open('models/RF/vectorizer.bin', 'wb'))
vectorizer = pickle.load(open('models/RF/vectorizer.bin', 'rb'))

In [ ]:
from imblearn.ensemble import BalancedRandomForestClassifier
model = BalancedRandomForestClassifier(max_depth=20, min_impurity_decrease = 0.001, 
                               criterion = "entropy", n_estimators=1000,  random_state=0)
model = model.fit(X_train,y_train)

In [77]:
evaluate(model.predict(X_test), y_test)

accuracy: 0.4065656565656566
F1-micro: 0.4065656565656566
F1-macro: 0.4127118842652152
Report:               precision    recall  f1-score   support

      center       0.30      0.21      0.24      1349
 center-left       0.30      0.26      0.28      1159
center-right       0.52      0.54      0.53      1754
    far-left       0.43      0.65      0.52       215
   far-right       0.41      0.62      0.49       671

    accuracy                           0.41      5148
   macro avg       0.39      0.46      0.41      5148
weighted avg       0.39      0.41      0.39      5148



In [6]:
y_train_bi = [(x in ['far-left', 'far-right']) for x in y_train]
y_test_bi = [(x in ['far-left', 'far-right']) for x in y_test]

In [24]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(max_depth=30, min_impurity_decrease = 0.001, 
                               criterion = "entropy", n_estimators=1000, 
                               class_weight='balanced_subsample', random_state=0)
model = model.fit(X_train,y_train_bi)

In [25]:
evaluate(model.predict(X_test), y_test_bi)

accuracy: 0.8578088578088578
F1-micro: 0.857808857808858
F1-macro: 0.7867275789626196
Report:               precision    recall  f1-score   support

       False       0.96      0.87      0.91      4262
        True       0.56      0.81      0.66       886

    accuracy                           0.86      5148
   macro avg       0.76      0.84      0.79      5148
weighted avg       0.89      0.86      0.87      5148



# EasyEnsemble

In [78]:
from imblearn.ensemble import EasyEnsembleClassifier
model = EasyEnsembleClassifier(n_estimators=10,  random_state=0)
model = model.fit(X_train,y_train)

In [79]:
evaluate(model.predict(X_test), y_test)

accuracy: 0.37975912975912973
F1-micro: 0.37975912975912973
F1-macro: 0.4069889587469615
Report:               precision    recall  f1-score   support

      center       0.38      0.75      0.50      1349
 center-left       0.24      0.14      0.17      1159
center-right       0.27      0.12      0.16      1754
    far-left       0.87      0.45      0.59       215
   far-right       0.52      0.73      0.61       671

    accuracy                           0.38      5148
   macro avg       0.46      0.43      0.41      5148
weighted avg       0.35      0.38      0.33      5148



# Logistic regression

In [72]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty = 'l2', C = 0.01, solver = 'saga', random_state=0)
model = model.fit(X_train,y_train)

In [73]:
evaluate(model.predict(X_test), y_test)

accuracy: 0.44716394716394714
F1-micro: 0.44716394716394714
F1-macro: 0.26213295697953276
Report:               precision    recall  f1-score   support

      center       0.41      0.20      0.27      1349
 center-left       0.37      0.49      0.42      1159
center-right       0.50      0.84      0.62      1754
    far-left       0.00      0.00      0.00       215
   far-right       0.00      0.00      0.00       671

    accuracy                           0.45      5148
   macro avg       0.25      0.30      0.26      5148
weighted avg       0.36      0.45      0.38      5148



/home/daksenov/miniconda2/envs/summ/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Naive Bayes

In [54]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB(alpha = 0.4)
model = model.fit(X_train,y_train)

In [55]:
evaluate(model.predict(X_test), y_test)

accuracy: 0.4020979020979021
F1-micro: 0.4020979020979021
F1-macro: 0.22343815569135422
Report:               precision    recall  f1-score   support

      center       0.42      0.12      0.19      1349
 center-left       0.31      0.42      0.35      1159
center-right       0.45      0.81      0.58      1754
    far-left       0.00      0.00      0.00       215
   far-right       0.00      0.00      0.00       671

    accuracy                           0.40      5148
   macro avg       0.24      0.27      0.22      5148
weighted avg       0.33      0.40      0.33      5148



/home/daksenov/miniconda2/envs/summ/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
